In [0]:
dbutils.fs.mounts()

In [0]:
dbutils.fs.unmount("/mnt/j2dcontainer")

In [0]:
dbutils.fs.mounts()

In [0]:
storage_account = "aldlsj2dtechstorage"
container_name = "j2dadlscontainer"
mount_point = "/mnt/my_adls_container"

# read the secrets from keyValut
client_id = dbutils.secrets.get(scope="sc-jd2tech-05", key="adb-05-client-id")
tenent_id = dbutils.secrets.get(scope="sc-jd2tech-05", key="adb-05-tenent-id")
secret = dbutils.secrets.get(scope="sc-jd2tech-05", key="adb-05-secret-id")

In [0]:
print(secret)

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": client_id,
    "fs.azure.account.oauth2.client.secret": secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenent_id}/oauth2/token"
}

In [0]:
dbutils.fs.mount(
  source = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/",
  mount_point = mount_point,
  extra_configs = configs
)

In [0]:
dbutils.fs.mounts()

In [0]:
dbutils.fs.ls("/mnt/my_adls_container")

In [0]:
%fs
ls /mnt/my_adls_container

In [0]:
%fs
ls dbfs:/mnt/my_adls_container/sql_tables/

In [0]:
product_df = spark.read.format("delta").load("dbfs:/mnt/my_adls_container/sql_tables/product_2025-07-03/")
product_df.display()

In [0]:
from pyspark.sql.types import DateType
from pyspark.sql import functions as F

In [0]:
# product_df = (
#     product_df
#     .withColumn("SellStartDate", F.col("SellStartDate").cast(DateType()))
#     .withColumn("SellEndDate", F.col("SellEndDate").cast(DateType()))
# )

In [0]:
product_df = (
    product_df
    .withColumns({
        "SellStartDate":  F.col("SellStartDate").cast(DateType()),
        "SellEndDate":  F.col("SellEndDate").cast(DateType())
    })
    .drop("ThumbNailPhoto")
)

display(product_df)

In [0]:
# write this dataframe into a csv file appending with date

In [0]:
from datetime import date

In [0]:
today_date = (date.today()).strftime("%d_%m_%Y")
today_date

In [0]:
product_df.write.format("csv").mode("overwrite").option("header", True).save(f"dbfs:/mnt/my_adls_container/samples/product_{today_date}")

In [0]:
product_df.rdd.getNumPartitions()

In [0]:
product_df = product_df.repartition(2)
product_df.rdd.getNumPartitions()

In [0]:
product_df.write.format("csv").mode("overwrite").option("header", True).save(f"dbfs:/mnt/my_adls_container/samples/product_{today_date}")

In [0]:
# write same dataframe into parquet file

product_df.write.format("parquet").mode("overwrite").save(f"dbfs:/mnt/my_adls_container/samples/product_parquet_{today_date}")

In [0]:
product_df.write.format("delta").mode("overwrite").save(f"dbfs:/mnt/my_adls_container/samples/product_delta_{today_date}")